In [2]:
import tensorflow as tf
import numpy as np
import C3D_model
import time
import data_processing
import os
import os.path
from os.path import join
import pickle
import tensorflow.contrib.slim as slim

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
TRAIN_LOG_DIR = os.path.join('Log/train/', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
# pretrained checkpoint 디렉토리 
TRAIN_CHECK_POINT = 'check_point/pretrained_C3D_checkpoint/'
# Non-local block 추가된 C3D 모델 checkpoint 저장 위치
NL_CHECK_POINT = 'check_point_nl/'
BATCH_SIZE = 10
NUM_CLASSES = 11
CROP_SZIE = 112
CHANNEL_NUM = 3
CLIP_LENGTH = 16
EPOCH_NUM = 20
LEARNING_RATE = 1e-5

## 1) Load UCF11(UCF YouTube Action) Dataset Path

In [3]:
DATA_SPLIT_PATH = 'data_split.pkl'
ucf11_dataset = pickle.load(open(DATA_SPLIT_PATH,'rb'))
train_set = ucf11_dataset['train']
test_set = ucf11_dataset['test']

## 2) Get Shuffle Index

In [4]:
train_video_indices = data_processing.get_video_indices(len(train_set))
test_video_indices = data_processing.get_video_indices(len(test_set))

## 3) Set Graph 

In [5]:

# Define graph
    
batch_clips = tf.placeholder(tf.float32, [BATCH_SIZE, CLIP_LENGTH, CROP_SZIE, CROP_SZIE, CHANNEL_NUM], name='X')
batch_labels = tf.placeholder(tf.int32, [BATCH_SIZE, NUM_CLASSES], name='Y')
keep_prob = tf.placeholder(tf.float32)
logits = C3D_model.C3D(batch_clips, NUM_CLASSES, keep_prob, non_local=True)

include_layers = ['C3D/conv1','C3D/conv2','C3D/conv3','C3D/conv4','C3D/conv5','C3D/fc6','C3D/fc7']
variables_to_restore = slim.get_variables_to_restore(include=include_layers)

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=batch_labels))
    tf.summary.scalar('entropy_loss', loss)

with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(batch_labels, 1)), np.float32))
    tf.summary.scalar('accuracy', accuracy)

optimizer = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)
saver = tf.train.Saver()
restorer = tf.train.Saver(variables_to_restore)
summary_op = tf.summary.merge_all()
    
    
    

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### Load Pretrained Weight of C3D

In [6]:
# Load pretrained weight of C3D
pretrained_path = tf.train.latest_checkpoint(TRAIN_CHECK_POINT)

### Training C3D + Non_local Block

In [7]:
# Set graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    train_summary_writer = tf.summary.FileWriter(TRAIN_LOG_DIR, sess.graph)
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    if pretrained_path != 0:
        restorer.restore(sess, pretrained_path)
        print('Pretrained C3D Model is restored')
        step = 0
    for epoch in range(EPOCH_NUM):
        accuracy_epoch = 0
        loss_epoch = 0
        batch_index = 0
        for i in range(len(train_video_indices) // BATCH_SIZE):
            step += 1
            # Get Batch for Training
            batch_data, batch_index = data_processing.get_batches(train_set, NUM_CLASSES, batch_index,
                                                         train_video_indices, BATCH_SIZE)
                
            # Train
            _, loss_out, accuracy_out, summary = sess.run([optimizer, loss, accuracy, summary_op],
                                                        feed_dict={batch_clips:batch_data['clips'],
                                                        batch_labels:batch_data['labels'],
                                                        keep_prob: 0.5})
            loss_epoch += loss_out
            accuracy_epoch += accuracy_out

            if i % 10 == 0:
                print('Epoch %d, Batch %d: Loss is %.5f; Accuracy is %.5f'%(epoch+1, i, loss_out, accuracy_out))
                train_summary_writer.add_summary(summary, step)

        print('Epoch %d: Average loss is: %.5f; Average accuracy is: %.5f'%(epoch+1, loss_epoch / (len(train_video_indices) // BATCH_SIZE),
                                                                                accuracy_epoch / (len(train_video_indices) // BATCH_SIZE)))
        accuracy_epoch = 0
        loss_epoch = 0
        batch_index = 0
            
        # Get validation results
        for i in range(len(test_video_indices) // BATCH_SIZE):
            batch_data, batch_index = data_processing.get_batches(test_set, NUM_CLASSES, batch_index,
                                                                      test_video_indices, BATCH_SIZE)
            loss_out, accuracy_out = sess.run([loss, accuracy],
                                                  feed_dict={batch_clips:batch_data['clips'],
                                                             batch_labels:batch_data['labels'],
                                                            keep_prob: 1.0})
            loss_epoch += loss_out
            accuracy_epoch += accuracy_out

        print('Test loss is %.5f; Accuracy is %.5f'%(loss_epoch / (len(test_video_indices) // BATCH_SIZE),
                                                               accuracy_epoch /(len(test_video_indices) // BATCH_SIZE)))
        saver.save(sess, NL_CHECK_POINT + 'c3d_nonlocal_ckpt', global_step=epoch)

INFO:tensorflow:Restoring parameters from check_point/c3d_ckpt-49
Pretrained C3D Model is restored
Epoch 1, Batch 0: Loss is 4.49675; Accuracy is 0.10000
Epoch 1, Batch 10: Loss is 2.40472; Accuracy is 0.20000
Epoch 1, Batch 20: Loss is 2.02235; Accuracy is 0.20000
Epoch 1, Batch 30: Loss is 1.42295; Accuracy is 0.80000
Epoch 1, Batch 40: Loss is 1.74381; Accuracy is 0.50000
Epoch 1, Batch 50: Loss is 1.17496; Accuracy is 0.60000
Epoch 1, Batch 60: Loss is 0.89018; Accuracy is 0.80000
Epoch 1, Batch 70: Loss is 0.66083; Accuracy is 0.80000
Epoch 1, Batch 80: Loss is 0.58907; Accuracy is 0.80000
Epoch 1, Batch 90: Loss is 0.39191; Accuracy is 0.90000
Epoch 1, Batch 100: Loss is 0.18499; Accuracy is 0.90000
Epoch 1, Batch 110: Loss is 0.04561; Accuracy is 1.00000
Epoch 1: Average loss is: 1.29836; Average accuracy is: 0.58860
Test loss is 2.60548; Accuracy is 0.49688
Epoch 2, Batch 0: Loss is 0.04417; Accuracy is 1.00000
Epoch 2, Batch 10: Loss is 0.30310; Accuracy is 0.90000
Epoch 2, Ba

Epoch 11, Batch 60: Loss is 0.35618; Accuracy is 0.80000
Epoch 11, Batch 70: Loss is 0.74758; Accuracy is 0.80000
Epoch 11, Batch 80: Loss is 0.17164; Accuracy is 0.90000
Epoch 11, Batch 90: Loss is 0.08960; Accuracy is 0.90000
Epoch 11, Batch 100: Loss is 0.42592; Accuracy is 0.90000
Epoch 11, Batch 110: Loss is 0.34918; Accuracy is 0.80000
Epoch 11: Average loss is: 0.26945; Average accuracy is: 0.93246
Test loss is 3.77602; Accuracy is 0.51563
Epoch 12, Batch 0: Loss is 0.81204; Accuracy is 0.90000
Epoch 12, Batch 10: Loss is 0.02751; Accuracy is 1.00000
Epoch 12, Batch 20: Loss is 0.53089; Accuracy is 0.90000
Epoch 12, Batch 30: Loss is 0.31686; Accuracy is 0.90000
Epoch 12, Batch 40: Loss is 0.24730; Accuracy is 0.90000
Epoch 12, Batch 50: Loss is 0.00042; Accuracy is 1.00000
Epoch 12, Batch 60: Loss is 0.00023; Accuracy is 1.00000
Epoch 12, Batch 70: Loss is 0.34983; Accuracy is 0.90000
Epoch 12, Batch 80: Loss is 0.21428; Accuracy is 0.90000
Epoch 12, Batch 90: Loss is 0.02706; 